In [1]:
import pandas as pd
import numpy as np

In [2]:
# IMPORTAÇÃO DA BASE DE SÓCIOS
socios_empresas_rotuladas_df = pd.read_csv(r'C:\DATASETS\Empresas Rotuladas\socios_empresas_rotuladas.csv', dtype=str)
socios_empresas_rotuladas_df

,cnpj,cnpj_cpf_socio,nome_socio,tempo_de_sociedade,cnpj_cpf_socio_mascarado
0,00000000000191,***923641**,MARCIO HAMILTON FERREIRA,11,***923641**
1,00000000000191,***491386**,NILSON MARTINIANO MOREIRA,11,***491386**
2,00000000000191,***718468**,WALTER MALIENI JUNIOR,11,***718468**
3,00000000000191,***415907**,CARLOS ALBERTO ARAUJO NETTO,9,***415907**
4,00000000000191,***022878**,ANTONIO MAURICIO MAURANO,9,***022878**
...,...,...,...,...,...
1683063,97541975000169,26090628859,EDIVALDO ALVES DE ARAUJO,10,***906288**
1683064,97542225000101,01095040707,CARLOS HENRIQUE ROCHA DE SOUZA,10,***950407**
1683065,97543869000114,72399317491,MARIO JORGE LIMA DA SILVA,10,***993174**
1683066,97548003000104,03028898330,CARLOS ADRIANO ROSA DA SILVA,10,***288983**


In [3]:
# Remover coluna de tempo_de_sociedade, não necessária aqui
socios_empresas_rotuladas_df.drop(columns=['tempo_de_sociedade'], inplace=True)

### Junção com a base de devedores da dívida ativa da União:

In [4]:
# IMPORTAÇÃO DA BASE DE DÍVIDA ATIVA
divida_ativa_df = pd.read_csv(r'C:\DATASETS\Divida_Ativa\divida_ativa_todos.csv', dtype=str)
divida_ativa_df

,CPF_CNPJ,NOME_DEVEDOR
0,***464712**,ERISVANDO TORQUATO DO NASCIMENTO
1,04718079000141,I. B. SOUSA
2,***468172**,CLAUDIO BRANDAO DA SILVA
3,05395942000130,DISTRIBUIDORA VALE DO GUAPORE LTDA
4,***117482**,SEBASTIAO PEREIRA DO NASCIMENTO
...,...,...
5994589,11966821000101,WANDERVAL APARECIDO DE FARIA
5994590,38135935000158,MIL TOPOGRAFIA E CONSULTORIA LTDA
5994591,13758946000163,ROSANA MESQUITA FERNANDES
5994592,12967577000156,AM E FILHOS COMERCIO DE MATERIAIS PARA CONSTRU...


In [5]:
print(divida_ativa_df.CPF_CNPJ.str.len().value_counts())
pj_devedores_df = divida_ativa_df[divida_ativa_df.CPF_CNPJ.str.len()==14]
pf_devedores_df = divida_ativa_df[divida_ativa_df.CPF_CNPJ.str.len()==11]

14    3089415
11    2905179
Name: CPF_CNPJ, dtype: int64


In [6]:
# Rotula os sócios que são PJ
socios_empresas_rotuladas_df['socio_devedor_uniao'] = (socios_empresas_rotuladas_df.merge(pj_devedores_df, left_on='cnpj_cpf_socio', right_on='CPF_CNPJ', how='left', indicator=True).eval('_merge == "both"').astype(bool))
socios_empresas_rotuladas_df.groupby(socios_empresas_rotuladas_df.cnpj_cpf_socio.str.len()).socio_devedor_uniao.value_counts()

cnpj_cpf_socio  socio_devedor_uniao
11              False                  1664526
14              False                    15514
                True                      3028
Name: socio_devedor_uniao, dtype: int64

Agora, vamos rotular as pessoas físicas:

Todos os CPFs já estão mascarados, como visto abaixo:

In [15]:
pf_devedores_df[(pf_devedores_df.CPF_CNPJ.str[0:1] != '*')]

,CPF_CNPJ,NOME_DEVEDOR


In [16]:
# Rotula os sócios que são PF
socios_empresas_rotuladas_df['socio_devedor_uniao_pf'] = (socios_empresas_rotuladas_df.merge(pf_devedores_df, left_on=['cnpj_cpf_socio_mascarado', 'nome_socio'], right_on=['CPF_CNPJ', 'NOME_DEVEDOR'], how='left', indicator=True).eval('_merge == "both"').astype(bool))
socios_empresas_rotuladas_df.groupby(socios_empresas_rotuladas_df.cnpj_cpf_socio.str.len()).socio_devedor_uniao_pf.value_counts()

cnpj_cpf_socio  socio_devedor_uniao_pf
11              False                     1619561
                True                        44965
14              False                       18542
Name: socio_devedor_uniao_pf, dtype: int64

In [17]:
# Junta os marcadores:
socios_empresas_rotuladas_df['socio_devedor_uniao'] = socios_empresas_rotuladas_df['socio_devedor_uniao']|socios_empresas_rotuladas_df['socio_devedor_uniao_pf']
socios_empresas_rotuladas_df.drop(columns=['socio_devedor_uniao_pf'], inplace=True)
socios_empresas_rotuladas_df.groupby(socios_empresas_rotuladas_df.cnpj_cpf_socio.str.len()).socio_devedor_uniao.value_counts()

cnpj_cpf_socio  socio_devedor_uniao
11              False                  1619561
                True                     44965
14              False                    15514
                True                      3028
Name: socio_devedor_uniao, dtype: int64

In [18]:
# Salvando resultado:
socios_empresas_rotuladas_df.to_csv(r'C:\DATASETS\Empresas Rotuladas\socios_empresas_rotuladas-devedores_da_uniao.csv', index = False)